In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
client = OpenAI(
    api_key=os.environ.get("RUN_POD_TOKEN"),
    base_url=f"https://api.runpod.ai/v2/b1lfpriy7k3ynw/openai/v1",
)
model_name= os.getenv("RUN_POD_MODEL_NAME")

client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the purpose of life?"},
    ],
    temperature=0.0,
    top_p=0.8,
    max_tokens=2000,
)